In [9]:
# Import relevant tools for NLP and load the english language library
import spacy
nlp = spacy.load('en_core_web_sm')
import numpy as np
import pandas as pd
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/Kirsten/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [10]:
# Import the relevant tools to get text from a website
from bs4 import BeautifulSoup
import requests

In [109]:
#Location of website and getting the content with BeautifulSoup
#url = "https://hbr.org/2020/01/the-upside-of-being-an-underdog"
url="https://hbr.org/2020/01/making-stakeholder-capitalism-a-reality"
webpage = requests.get(url)
content_page = webpage.content
soup = BeautifulSoup(content_page, 'html.parser')
title = []
paragraphtext = []    
 
# get article title
try:
    abody = soup.find('title')
    title = abody.get_text() 
except:
    title = 'Anonymous'
## get main article text
articlebody = soup.find(class_='article article-first-row').find_all('p')
#put main body of text into a list with the text structured paragraph by paragraph
for paragraph in articlebody[:]:
        # get the text only
    text = paragraph.get_text()
    paragraphtext.append(text)        
    
#print(title)
#print(paragraphtext)

In [110]:
input_text = nlp(paragraphtext[5])

In [111]:
#read in subtlexus corpus, large corpus with frequency information
corpus = pd.read_csv('SUBTLEXusExcel2007.tsv', sep='\t',index_col ="Word")
corpus.head()

,FREQcount,CDcount,FREQlow,Cdlow,SUBTLWF,Lg10WF,SUBTLCD,Lg10CD
Word,,,,,,,,
the,1501908,8388,1339811,8388,29449.18,6.1766,100.00,3.9237
to,1156570,8383,1138435,8380,22677.84,6.0632,99.94,3.9235
a,1041179,8382,976941,8380,20415.27,6.0175,99.93,3.9234
you,2134713,8381,1595028,8376,41857.12,6.3293,99.92,3.9233
and,682780,8379,515365,8374,13387.84,5.8343,99.89,3.9232


In [112]:
#for those word types carrying meaning, make a ranking according to frequency
listofwordtypes=['VERB','ADV','NOUN','ADJ']
counter=0
corpus_results=[]
corpus_results_counter=[]
for token in input_text: 
    if (token.pos_ in listofwordtypes):
        try:
            corpus_results.append(corpus.loc[token.lemma_,'Lg10WF'])
            corpus_results_counter.append(counter)
        except KeyError:
              pass
        #  print('x')
        # print('x')
    #  #  print(corpus.loc[token.lemma_,'Lg10WF'])#,corpus.loc[token.lemma_].Lg10WF)
    counter=counter+1 
outputtuple=sorted(zip(corpus_results, corpus_results_counter))[:3]    
print(input_text[outputtuple[0][1]],input_text[outputtuple[1][1]],input_text[outputtuple[2][1]])

wickedly critical reality


In [113]:
from nltk.corpus import wordnet as wn  #Import wordnet from the NLTK
syn=[]
for thing in outputtuple:
    for synset in wn.synsets(str(input_text[thing[1]])):
        for lemma in synset.lemmas():
            #print((input_text[thing[1]]))
            #print(lemma.name())
            if str(lemma.name()) != str(input_text[thing[1]]):
                syn.append(lemma.name())    #add the synonyms

print(syn)

['evilly', 'vital', 'decisive', 'world', 'realness', 'realism']


In [101]:
#import tools for putting information into a database
import sqlite3
from sqlite3 import Error

In [102]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)
 
    return conn

In [106]:
def create_TextFromWebsites_table(conn, website):
    """
    Create a new project into the projects table
    :param conn:
    :param project:
    :return: project id
    """
    sql = ''' INSERT INTO websitetext(name,url,nopara,para1,para2,para3,para4)
              VALUES(?,?,?,?,?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, website)
    return cur.lastrowid

In [107]:
def main():
    database = r"/Users/Kirsten/InsightProjectLocalStuff/Ideas/MVP/Database/insightproject.db"
  # create a database connection
    conn = create_connection(database)
    with conn:
        website_2=(title,url,len(paragraphtext),paragraphtext[0],paragraphtext[1],paragraphtext[2],paragraphtext[3])
        create_TextFromWebsites_table(conn,website_2)

In [108]:
if __name__ == '__main__':
    main()

In [67]:
len(paragraphtext)

17